In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
import numpy as np
import pyprind
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import pprint
import copy
import wurst as w
import brightway2 as bw
import bw2analyzer as bwa

from brightway2 import *
from matplotlib import pyplot as plt
from pandas import DataFrame
from openpyxl import load_workbook
from wurst import *
from wurst.searching import *
from wurst.transformations.activity import change_exchanges_by_constant_factor
from wurst.transformations.uncertainty import rescale_exchange
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group
from wurst.IMAGE.io import *
from wurst.IMAGE import *
from wurst.ecoinvent.electricity_markets import *
from wurst.ecoinvent.filters import *
from wurst.transformations.geo import *

In [3]:
%run 1_Functions_Market_electricity_data.ipynb

## 1. LCA calculation
##### In this step, we calculate the LCA results of two DAC technologies at each year (under each scenario)

In [4]:
projects.set_current("DAC_LCA")

In [5]:
list(databases)

[]

In [447]:
# Here we need to change the database_name variable when we calculate the LCA result at a specific year + scenario combination. 

database_name = 'ecoinvent_2020_RCP19_DAC'
ei_cf_36_db = bw.Database(database_name)
    
# Creating new processes for solvent- and sorbent-based DACCS
# Solvent Process

Solvent_construction = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_construction', name = "Solvent construction", unit = "ton")
Solvent_operation_material = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_operation_material', name = "Solvent operation material", unit = "ton")
 
Solvent_operation_electriciy_CN = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_operation_electriciy_CN', name = "Solvent operation electriciy CN", unit = "kWh")
Solvent_operation_electriciy_US = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_operation_electriciy_US', name = "Solvent operation electriciy US", unit = "kWh")
Solvent_operation_electriciy_EU = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_operation_electriciy_EU', name = "Solvent operation electriciy EU", unit = "kWh")
Solvent_operation_electriciy_RU = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_operation_electriciy_RU', name = "Solvent operation electriciy RU", unit = "kWh")
Solvent_operation_electriciy_GL = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_operation_electriciy_GL', name = "Solvent operation electriciy GL", unit = "kWh")

Solvent_operation_heat_NG = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_operation_heat_NG', name = "Solvent operation heat NG", unit = "MJ")
Solvent_operation_heat_biogas = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_operation_heat_biogas', name = "Solvent operation heat biogas", unit = "MJ")

Solvent_EoL_treat = ei_cf_36_db.new_activity(code = 'DAC_LCA_Solvent_EoL_treatment', name = "Solvent end-of-life treatment", unit = "ton")


# Sorbent Process

Sorbent_construction = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_construction', name = "Sorbent construction", unit = "ton")
Sorbent_operation_material = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_material', name = "Sorbent operation material", unit = "ton")

Sorbent_operation_electriciy_CN = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_electriciy_CN', name = "Sorbent operation electriciy CN", unit = "kWh")
Sorbent_operation_electriciy_US = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_electriciy_US', name = "Sorbent operation electriciy US", unit = "kWh")
Sorbent_operation_electriciy_EU = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_electriciy_EU', name = "Sorbent operation electriciy EU", unit = "kWh")
Sorbent_operation_electriciy_RU = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_electriciy_RU', name = "Sorbent operation electriciy RU", unit = "kWh")
Sorbent_operation_electriciy_GL = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_electriciy_GL', name = "Sorbent operation electriciy GL", unit = "kWh")

Sorbent_operation_heat_pump_CN_elec = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_heat_pump_CN_elec', name = "Sorbent operation heat pump CN elec", unit = "MJ")
Sorbent_operation_heat_pump_US_elec = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_heat_pump_US_elec', name = "Sorbent operation heat pump US elec", unit = "MJ")
Sorbent_operation_heat_pump_EU_elec = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_heat_pump_EU_elec', name = "Sorbent operation heat pump EU elec", unit = "MJ")
Sorbent_operation_heat_pump_RU_elec = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_heat_pump_RU_elec', name = "Sorbent operation heat pump RU elec", unit = "MJ")
Sorbent_operation_heat_pump_GL_elec = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_heat_pump_GL_elec', name = "Sorbent operation heat pump GL elec", unit = "MJ")

Sorbent_operation_heat_biogas = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_operation_heat_biogas', name = "Sorbent operation heat biogas", unit = "MJ")

Sorbent_EoL_treat = ei_cf_36_db.new_activity(code = 'DAC_LCA_Sorbent_EoL_treatment', name = "Sorbent end-of-life treatment", unit = "ton")


In [448]:
# extract the activities in the LCI database and assign them with a name

# Construction
Concrete = [act for act in ei_cf_36_db if 'market for concrete, normal' == act['name'] and 'RoW' in act['location']][0]
Sand = [act for act in ei_cf_36_db if 'market for sand' == act['name'] and 'RoW' == act['location']][0]
Refractory_brick = [act for act in ei_cf_36_db if 'market for refractory, basic, packed' == act['name'] and 'GLO' == act['location']][0]
Low_alloyed_steel = [act for act in ei_cf_36_db if 'market for steel, low-alloyed' in act['name'] and 'GLO' in act['location']][0]
Stainless_steel = [act for act in ei_cf_36_db if 'steel production, chromium steel 18/8, hot rolled' == act['name'] and 'RoW' == act['location']][0]
Reinforcing_steel = [act for act in ei_cf_36_db if 'market for reinforcing steel' == act['name'] and 'GLO' == act['location']][0]
Un_alloyed_steel = [act for act in ei_cf_36_db if 'market for steel, unalloyed' == act['name'] and 'GLO' == act['location']][0]
Steel_pipes = [act for act in ei_cf_36_db if 'market for drawing of pipe, steel' == act['name'] and 'GLO' == act['location']][0]
Copper = [act for act in ei_cf_36_db if 'market for copper' == act['name'] and 'GLO' == act['location']][0]
Aluminium = [act for act in ei_cf_36_db if 'market for aluminium, wrought alloy' == act['name'] and 'GLO' == act['location']][0]
Polypropylene = [act for act in ei_cf_36_db if 'market for polypropylene, granulate' == act['name'] and 'GLO' in act['location']][0]
Polyurethane = [act for act in ei_cf_36_db if 'market for polyurethane, flexible foam' == act['name'] and 'RoW' == act['location']][0]
Polyvinylchloride_PVC = [act for act in ei_cf_36_db if 'market for polyvinylchloride, bulk polymerised' == act['name'] and 'GLO' == act['location']][0]
Polyethylene = [act for act in ei_cf_36_db if 'market for polyethylene, low density, granulate' == act['name'] and 'GLO' == act['location']][0]
Glass_fiber = [act for act in ei_cf_36_db if 'market for glass fibre' == act['name'] and 'GLO' == act['location']][0]
Diesel_heavy_fuel_oil = [act for act in ei_cf_36_db if 'market for diesel' == act['name'] and 'RoW' == act['location']][0]
Bitumen = [act for act in ei_cf_36_db if 'market for bitumen seal' == act['name'] and 'GLO' == act['location']][0]
Transport = [act for act in ei_cf_36_db if 'market for transport, freight, lorry, unspecified' == act['name'] and 'RoW' == act['location']][0]
Well_construction = [act for act in ei_cf_36_db if 'market for onshore well, oil/gas' == act['name'] and 'GLO' == act['location']][0]
Stone_wool = [act for act in ei_cf_36_db if 'market for stone wool' == act['name'] and 'GLO' == act['location']][0]
Plastics_TPE = [act for act in ei_cf_36_db if 'market for polyurethane, rigid foam' == act['name'] and 'RoW' == act['location']][0]
Paint = [act for act in ei_cf_36_db if 'market for alkyd paint, white, without solvent, in 60% solution state' == act['name'] and 'RoW' == act['location']][0]
Plastic_insulation = [act for act in ei_cf_36_db if 'market for polystyrene foam slab for perimeter insulation' == act['name'] and 'GLO' == act['location']][0]


# Material
CaCO3 = [act for act in ei_cf_36_db if 'market for limestone, crushed, for mill' == act['name'] and 'RoW' == act['location']][0]
KOH = [act for act in ei_cf_36_db if 'market for potassium hydroxide' == act['name'] and 'GLO' == act['location']][0]
Water = [act for act in ei_cf_36_db if 'market for tap water' == act['name'] and 'RoW' == act['location']][0]
CW_amine_sorbent = [act for act in ei_cf_36_db if 'production of amine on silica' == act['name'] and 'GLO' == act['location']][0]

# Energy
NG_heat_CE = [act for act in ei_cf_36_db if 'heat production, natural gas, at industrial furnace >100kW' == act['name'] and 'RoW' == act['location']][0]
Biogas_heat = [act for act in ei_cf_36_db if 'heat production, biomethane, at boiler condensing modulating <100kW' == act['name'] and 'RoW' in act['location']][0]

Electricity_US = [act for act in ei_cf_36_db if 'market group for electricity, medium voltage' == act['name'] and 'US' == act['location']][0]
Electricity_CN = [act for act in ei_cf_36_db if 'market group for electricity, medium voltage' == act['name'] and 'CN' == act['location']][0]
Electricity_EU = [act for act in ei_cf_36_db if 'market group for electricity, medium voltage' == act['name'] and 'ENTSO-E' == act['location']][0]
Electricity_RU = [act for act in ei_cf_36_db if 'market for electricity, medium voltage' == act['name'] and 'RU' == act['location']][0]
Electricity_GL = [act for act in ei_cf_36_db if 'market group for electricity, medium voltage' == act['name'] and 'GLO' == act['location']][0]

# End of life treatment
treat_concrete_landfill = [act for act in ei_cf_36_db if 'treatment of waste concrete, inert material landfill' == act['name'] and 'RoW' == act['location']][0]
treat_steel_recycling = [act for act in ei_cf_36_db if 'treatment of waste reinforcement steel, recycling' == act['name'] and 'RoW' == act['location']][0]
treat_aluminium_recycling = [act for act in ei_cf_36_db if 'treatment of aluminium scrap, post-consumer, prepared for recycling, at remelter' == act['name'] and 'RoW' == act['location']][0]
treat_copper_incineration = [act for act in ei_cf_36_db if 'treatment of scrap copper, municipal incineration' == act['name'] and 'RoW' == act['location']][0]
treat_polyethylene_incineration = [act for act in ei_cf_36_db if 'treatment of waste polyethylene, municipal incineration' == act['name'] and 'RoW' == act['location'] and 'kilogram' == act['unit']][0]
treat_bitumen_landfill = [act for act in ei_cf_36_db if 'treatment of waste bitumen, sanitary landfill' == act['name'] and 'RoW' == act['location']][0]
treat_brick_sand_disposal = [act for act in ei_cf_36_db if 'treatment of waste brick, collection for final disposal' == act['name'] and 'RoW' == act['location']][0]
treat_polyurethane_incineration = [act for act in ei_cf_36_db if 'treatment of waste polyurethane, municipal incineration' == act['name'] and 'RoW' == act['location'] and 'kilogram' == act['unit']][0]
treat_polypropylene_incineration = [act for act in ei_cf_36_db if 'treatment of waste polypropylene, municipal incineration' == act['name'] and 'RoW' == act['location'] and 'kilogram' == act['unit']][0]
treat_polyvinylchloride_incineration = [act for act in ei_cf_36_db if 'treatment of waste polyvinylchloride, municipal incineration' == act['name'] and 'RoW' == act['location'] and 'kilogram' == act['unit']][0]
treat_plastic_mix_incineration = [act for act in ei_cf_36_db if 'treatment of waste plastic, mixture, municipal incineration' == act['name'] and 'RoW' == act['location'] and 'kilogram' == act['unit']][0]
treat_limestone_landfill = [act for act in ei_cf_36_db if 'treatment of limestone residue, inert material landfill' == act['name'] and 'RoW' == act['location']][0]
treat_sorbent_incineration = [act for act in ei_cf_36_db if 'treatment of spent anion exchange resin from potable water production, municipal incineration' == act['name'] and 'RoW' == act['location'] and 'kilogram' == act['unit']][0]
treat_solvent_incineration = [act for act in ei_cf_36_db if 'treatment of spent solvent mixture, hazardous waste incineration' == act['name'] and 'RoW' == act['location'] and 'kilogram' == act['unit']][0]
treat_stone_wool = [act for act in ei_cf_36_db if 'treatment of waste mineral wool, inert material landfill' == act['name'] and 'RoW' == act['location'] and 'kilogram' == act['unit']][0]


In [449]:
## Import the inventory data for both solvent- and sorbent-based DACCS
# "Solvent_import_LR_ave" and "Sorbent_import_LR_ave" tabs include the LCI inventory data of solvent- and sorbent-based DACCS
# from 2020 to 2100 developed based on reference learning rates. To calculate the LCA result under SSP2-RCP1.9-DACCS scenario
# without DACCS learning, we will just use the LCI data in the first column (representing 2020), while changing the background LCI database
# from 2020 to 2100. However, to calculate the LCA result udner SSP2-RCP1.9-DACCS scenario with DACCS learning, we need to use the dynamic
# LCI data from 2020 to 2100 together with the corresponding background LCI database from 2020 to 2100.

# "Solvent_import_LR_low" and "Sorbent_import_LR_low" tabs include the LCI inventory data of solvent- and sorbent-based DACCS
# from 2020 to 2100 developed based on slow learning rates.

# "Solvent_import_LR_high" and "Sorbent_import_LR_high" tabs include the LCI inventory data of solvent- and sorbent-based DACCS
# from 2020 to 2100 developed based on fast learning rates.


Solvent_inventory = pd.read_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\Ecoinvent\LCA_inventory_08162021.xlsx', sheet_name='Solvent_import_LR_ave')
Sorbent_inventory = pd.read_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\Ecoinvent\LCA_inventory_08162021.xlsx', sheet_name='Sorbent_import_LR_ave')

#Solvent_inventory = pd.read_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\Ecoinvent\LCA_inventory_08162021.xlsx', sheet_name='Solvent_import_LR_low')
#Sorbent_inventory = pd.read_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\Ecoinvent\LCA_inventory_08162021.xlsx', sheet_name='Sorbent_import_LR_low')

#Solvent_inventory = pd.read_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\Ecoinvent\LCA_inventory_08162021.xlsx', sheet_name='Solvent_import_LR_high')
#Sorbent_inventory = pd.read_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\Ecoinvent\LCA_inventory_08162021.xlsx', sheet_name='Sorbent_import_LR_high')


In [450]:
# column_i specify the year of LCI data that are being used. When technology learning is considered, 
# LCI data change over time, so we need to change this column_i variable and specify the year of LCI 
# data we want to import.

column_i = 3

# 3 -- 2020
# 4 -- 2030
# 5 -- 2040
# 6 -- 2050
# 7 -- 2060
# 8 -- 2070
# 9 -- 2080
# 10 -- 2090
# 11 -- 2100

In [451]:
## we need to delete the exchanges assigned to the processes when we run the scripts for different years

Solvent_construction.exchanges().delete()
Solvent_operation_material.exchanges().delete()

Solvent_operation_electriciy_CN.exchanges().delete()
Solvent_operation_electriciy_US.exchanges().delete()
Solvent_operation_electriciy_EU.exchanges().delete()
Solvent_operation_electriciy_RU.exchanges().delete()
Solvent_operation_electriciy_GL.exchanges().delete()

Solvent_operation_heat_NG.exchanges().delete()
Solvent_operation_heat_biogas.exchanges().delete()

Solvent_EoL_treat.exchanges().delete()

In [452]:
Sorbent_construction.exchanges().delete()
Sorbent_operation_material.exchanges().delete()

Sorbent_operation_electriciy_CN.exchanges().delete()
Sorbent_operation_electriciy_US.exchanges().delete()
Sorbent_operation_electriciy_EU.exchanges().delete()
Sorbent_operation_electriciy_RU.exchanges().delete()
Sorbent_operation_electriciy_GL.exchanges().delete()

Sorbent_operation_heat_biogas.exchanges().delete()
Sorbent_operation_heat_pump_CN_elec.delete()
Sorbent_operation_heat_pump_US_elec.delete()
Sorbent_operation_heat_pump_EU_elec.delete()
Sorbent_operation_heat_pump_RU_elec.delete()
Sorbent_operation_heat_pump_GL_elec.delete()

Sorbent_EoL_treat.exchanges().delete()

In [453]:
# assign material and energy use (both activities and their amount) to the processes

Sol_data_input = Solvent_inventory[Solvent_inventory.columns[[0, 1, 2, column_i]]]

## Construction
# Air_contact
Solvent_construction.new_exchange(input = Concrete.key, amount = Sol_data_input.iat[0, 3], unit = "cubic meter",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Low_alloyed_steel.key, amount = Sol_data_input.iat[1, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Stainless_steel.key, amount = Sol_data_input.iat[2, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Polyurethane.key, amount = Sol_data_input.iat[3, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Glass_fiber.key, amount = Sol_data_input.iat[4, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Polypropylene.key, amount = Sol_data_input.iat[5, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Polyvinylchloride_PVC.key, amount = Sol_data_input.iat[6, 3], unit = "kilogram",type = 'technosphere').save()
# Pellet_reactor
Solvent_construction.new_exchange(input = Concrete.key, amount = Sol_data_input.iat[7, 3], unit = "cubic meter",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Stainless_steel.key, amount = Sol_data_input.iat[8, 3], unit = "kilogram",type = 'technosphere').save()
# Calciner_slaker
Solvent_construction.new_exchange(input = Concrete.key, amount = Sol_data_input.iat[9, 3], unit = "cubic meter",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Low_alloyed_steel.key, amount = Sol_data_input.iat[10, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Refractory_brick.key, amount = Sol_data_input.iat[11, 3], unit = "kilogram",type = 'technosphere').save()
# Other_equipment
Solvent_construction.new_exchange(input = Concrete.key, amount = Sol_data_input.iat[12, 3], unit = "cubic meter",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Aluminium.key, amount = Sol_data_input.iat[13, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Low_alloyed_steel.key, amount = Sol_data_input.iat[14, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Stainless_steel.key, amount = Sol_data_input.iat[15, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Un_alloyed_steel.key, amount = Sol_data_input.iat[16, 3], unit = "kilogram",type = 'technosphere').save()

# Compression_transport_storage
Solvent_construction.new_exchange(input = Concrete.key, amount = Sol_data_input.iat[17, 3], unit = "cubic meter",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Copper.key, amount = Sol_data_input.iat[18, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Low_alloyed_steel.key, amount = Sol_data_input.iat[19, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Polyethylene.key, amount = Sol_data_input.iat[20, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Sand.key, amount = Sol_data_input.iat[21, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Reinforcing_steel.key, amount = Sol_data_input.iat[22, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Steel_pipes.key, amount = Sol_data_input.iat[23, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Bitumen.key, amount = Sol_data_input.iat[24, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Un_alloyed_steel.key, amount = Sol_data_input.iat[25, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Transport.key, amount = Sol_data_input.iat[26, 3], unit = "ton kilometer",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Diesel_heavy_fuel_oil.key, amount = Sol_data_input.iat[27, 3], unit = "megajoule",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Well_construction.key, amount = Sol_data_input.iat[28, 3], unit = "kilometer",type = 'technosphere').save()
Solvent_construction.new_exchange(input = Electricity_GL.key, amount = Sol_data_input.iat[29, 3], unit = "kilowatt hour",type = 'technosphere').save()

## Solvent_and_material
Solvent_operation_material.new_exchange(input = KOH.key, amount = Sol_data_input.iat[30, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_operation_material.new_exchange(input = CaCO3.key, amount = Sol_data_input.iat[31, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_operation_material.new_exchange(input = Water.key, amount = Sol_data_input.iat[32, 3], unit = "kilogram",type = 'technosphere').save()

## Electricity_consumption
Solvent_operation_electriciy_CN.new_exchange(input = Electricity_CN.key, amount = Sol_data_input.iat[33, 3], unit = "kilowatt hour",type = 'technosphere').save()
Solvent_operation_electriciy_US.new_exchange(input = Electricity_US.key, amount = Sol_data_input.iat[33, 3], unit = "kilowatt hour",type = 'technosphere').save()
Solvent_operation_electriciy_EU.new_exchange(input = Electricity_EU.key, amount = Sol_data_input.iat[33, 3], unit = "kilowatt hour",type = 'technosphere').save()
Solvent_operation_electriciy_RU.new_exchange(input = Electricity_RU.key, amount = Sol_data_input.iat[33, 3], unit = "kilowatt hour",type = 'technosphere').save()
Solvent_operation_electriciy_GL.new_exchange(input = Electricity_GL.key, amount = Sol_data_input.iat[33, 3], unit = "kilowatt hour",type = 'technosphere').save()

## Heat_consumption
Solvent_operation_heat_NG.new_exchange(input = NG_heat_CE.key, amount = Sol_data_input.iat[34, 3], unit = "megajoule",type = 'technosphere').save()
Solvent_operation_heat_biogas.new_exchange(input = Biogas_heat.key, amount = Sol_data_input.iat[34, 3], unit = "megajoule",type = 'technosphere').save()


## End-of-life treatment
Solvent_EoL_treat.new_exchange(input = treat_concrete_landfill.key, amount = Sol_data_input.iat[35, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_steel_recycling.key, amount = Sol_data_input.iat[36, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_aluminium_recycling.key, amount = Sol_data_input.iat[37, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_copper_incineration.key, amount = Sol_data_input.iat[38, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_polyethylene_incineration.key, amount = Sol_data_input.iat[39, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_bitumen_landfill.key, amount = Sol_data_input.iat[40, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_brick_sand_disposal.key, amount = Sol_data_input.iat[41, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_polyurethane_incineration.key, amount = Sol_data_input.iat[42, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_polypropylene_incineration.key, amount = Sol_data_input.iat[43, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_polyvinylchloride_incineration.key, amount = Sol_data_input.iat[44, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_plastic_mix_incineration.key, amount = Sol_data_input.iat[45, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_limestone_landfill.key, amount = Sol_data_input.iat[46, 3], unit = "kilogram",type = 'technosphere').save()
Solvent_EoL_treat.new_exchange(input = treat_solvent_incineration.key, amount = Sol_data_input.iat[47, 3], unit = "kilogram",type = 'technosphere').save()



In [322]:
# Save the processes after assigning the activity name and amount. This is only need to be done once for each database.
# Once the new processes are created in the database, we don't need to run these scripts again.

Solvent_construction.save()
Solvent_operation_material.save()

Solvent_operation_electriciy_CN.save()
Solvent_operation_electriciy_US.save()
Solvent_operation_electriciy_EU.save()
Solvent_operation_electriciy_RU.save()
Solvent_operation_electriciy_GL.save()

Solvent_operation_heat_NG.save()
Solvent_operation_heat_biogas.save()

Solvent_EoL_treat.save()

In [454]:
# assign material and energy use (both activities and their amount) to the processes

Sor_data_input = Sorbent_inventory[Sorbent_inventory.columns[[0, 1, 2, column_i]]]
## Construction
# Air_contact
Sorbent_construction.new_exchange(input = Un_alloyed_steel.key, amount = Sor_data_input.iat[0, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Stainless_steel.key, amount = Sor_data_input.iat[1, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Stone_wool.key, amount = Sor_data_input.iat[2, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Plastics_TPE.key, amount = Sor_data_input.iat[3, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Copper.key, amount = Sor_data_input.iat[4, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Aluminium.key, amount = Sor_data_input.iat[5, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Paint.key, amount = Sor_data_input.iat[6, 3], unit = "kilogram",type = 'technosphere').save()

# Other_equipment
Sorbent_construction.new_exchange(input = Concrete.key, amount = Sor_data_input.iat[7, 3], unit = "cubic meter",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Reinforcing_steel.key, amount = Sor_data_input.iat[8, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Low_alloyed_steel.key, amount = Sor_data_input.iat[9, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Stone_wool.key, amount = Sor_data_input.iat[10, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Stainless_steel.key, amount = Sor_data_input.iat[11, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Plastic_insulation.key, amount = Sor_data_input.iat[12, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Plastics_TPE.key, amount = Sor_data_input.iat[13, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Copper.key, amount = Sor_data_input.iat[14, 3], unit = "kilogram",type = 'technosphere').save()

# Compression_transport_storage
Sorbent_construction.new_exchange(input = Concrete.key, amount = Sor_data_input.iat[15, 3], unit = "cubic meter",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Copper.key, amount = Sor_data_input.iat[16, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Low_alloyed_steel.key, amount = Sor_data_input.iat[17, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Polyethylene.key, amount = Sor_data_input.iat[18, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Sand.key, amount = Sor_data_input.iat[19, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Reinforcing_steel.key, amount = Sor_data_input.iat[20, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Steel_pipes.key, amount = Sor_data_input.iat[21, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Bitumen.key, amount = Sor_data_input.iat[22, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Un_alloyed_steel.key, amount = Sor_data_input.iat[23, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Transport.key, amount = Sor_data_input.iat[24, 3], unit = "ton kilometer",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Diesel_heavy_fuel_oil.key, amount = Sor_data_input.iat[25, 3], unit = "megajoule",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Well_construction.key, amount = Sor_data_input.iat[26, 3], unit = "kilometer",type = 'technosphere').save()
Sorbent_construction.new_exchange(input = Electricity_GL.key, amount = Sor_data_input.iat[27, 3], unit = "kilowatt hour",type = 'technosphere').save()

## Sorbent
Sorbent_operation_material.new_exchange(input = CW_amine_sorbent, amount = Sor_data_input.iat[28, 3], unit = "kilogram",type = 'technosphere').save()

## Electricity_consumption
Sorbent_operation_electriciy_CN.new_exchange(input = Electricity_CN.key, amount = Sor_data_input.iat[29, 3], unit = "kilowatt hour",type = 'technosphere').save()
Sorbent_operation_electriciy_US.new_exchange(input = Electricity_US.key, amount = Sor_data_input.iat[29, 3], unit = "kilowatt hour",type = 'technosphere').save()
Sorbent_operation_electriciy_EU.new_exchange(input = Electricity_EU.key, amount = Sor_data_input.iat[29, 3], unit = "kilowatt hour",type = 'technosphere').save()
Sorbent_operation_electriciy_RU.new_exchange(input = Electricity_RU.key, amount = Sor_data_input.iat[29, 3], unit = "kilowatt hour",type = 'technosphere').save()
Sorbent_operation_electriciy_GL.new_exchange(input = Electricity_GL.key, amount = Sor_data_input.iat[29, 3], unit = "kilowatt hour",type = 'technosphere').save()

## Heat_consumption
Sorbent_operation_heat_biogas.new_exchange(input = Biogas_heat.key, amount = Sor_data_input.iat[30, 3], unit = "megajoule",type = 'technosphere').save()
Sorbent_operation_heat_pump_CN_elec.new_exchange(input = Electricity_CN.key, amount = Sor_data_input.iat[31, 3], unit = "kilowatt hour",type = 'technosphere').save()
Sorbent_operation_heat_pump_US_elec.new_exchange(input = Electricity_US.key, amount = Sor_data_input.iat[31, 3], unit = "kilowatt hour",type = 'technosphere').save()
Sorbent_operation_heat_pump_EU_elec.new_exchange(input = Electricity_EU.key, amount = Sor_data_input.iat[31, 3], unit = "kilowatt hour",type = 'technosphere').save()
Sorbent_operation_heat_pump_RU_elec.new_exchange(input = Electricity_RU.key, amount = Sor_data_input.iat[31, 3], unit = "kilowatt hour",type = 'technosphere').save()
Sorbent_operation_heat_pump_GL_elec.new_exchange(input = Electricity_GL.key, amount = Sor_data_input.iat[31, 3], unit = "kilowatt hour",type = 'technosphere').save()
## .....
## Can potentially add other heat sources

## End-of-life treatment
Sorbent_EoL_treat.new_exchange(input = treat_concrete_landfill.key, amount = Sor_data_input.iat[32, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_EoL_treat.new_exchange(input = treat_steel_recycling.key, amount = Sor_data_input.iat[33, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_EoL_treat.new_exchange(input = treat_aluminium_recycling.key, amount = Sor_data_input.iat[34, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_EoL_treat.new_exchange(input = treat_copper_incineration.key, amount = Sor_data_input.iat[35, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_EoL_treat.new_exchange(input = treat_stone_wool.key, amount = Sor_data_input.iat[36, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_EoL_treat.new_exchange(input = treat_bitumen_landfill.key, amount = Sor_data_input.iat[37, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_EoL_treat.new_exchange(input = treat_brick_sand_disposal.key, amount = Sor_data_input.iat[38, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_EoL_treat.new_exchange(input = treat_plastic_mix_incineration.key, amount = Sor_data_input.iat[39, 3], unit = "kilogram",type = 'technosphere').save()
Sorbent_EoL_treat.new_exchange(input = treat_sorbent_incineration.key, amount = Sor_data_input.iat[40, 3], unit = "kilogram",type = 'technosphere').save()


In [324]:
# Save the processes after assigning the activity name and amount. This is only need to be done once for each database.
# Once the new processes are created in the database, we don't need to run these scripts again.


Sorbent_construction.save()
Sorbent_operation_material.save()

Sorbent_operation_electriciy_CN.save()
Sorbent_operation_electriciy_US.save()
Sorbent_operation_electriciy_EU.save()
Sorbent_operation_electriciy_RU.save()
Sorbent_operation_electriciy_GL.save()

#Sorbent_operation_heat_NG.save()
Sorbent_operation_heat_biogas.save()
#Sorbent_operation_heat_geothermal.save()
Sorbent_operation_heat_pump_CN_elec.save()
Sorbent_operation_heat_pump_US_elec.save()
Sorbent_operation_heat_pump_EU_elec.save()
Sorbent_operation_heat_pump_RU_elec.save()
Sorbent_operation_heat_pump_GL_elec.save()

Sorbent_EoL_treat.save()

In [455]:
## Select the impact categories and defining the functional unit

method_key = [[m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'agricultural land occupation' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'climate change' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'fossil depletion' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'freshwater ecotoxicity' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'freshwater eutrophication' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'human toxicity' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'ionising radiation' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'marine ecotoxicity' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'marine eutrophication' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'metal depletion' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'natural land transformation' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'ozone depletion' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'particulate matter formation' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'photochemical oxidant formation' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'terrestrial acidification' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'terrestrial ecotoxicity' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'urban land occupation' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'water depletion' in str(m)][0]]

# Functional unit
# Solvent
Solvent_construction_functional_unit = {Solvent_construction:1}
Solvent_operation_material_functional_unit = {Solvent_operation_material:1}

Solvent_operation_electriciy_CN_functional_unit = {Solvent_operation_electriciy_CN:1}
Solvent_operation_electriciy_US_functional_unit = {Solvent_operation_electriciy_US:1}
Solvent_operation_electriciy_EU_functional_unit = {Solvent_operation_electriciy_EU:1}
Solvent_operation_electriciy_RU_functional_unit = {Solvent_operation_electriciy_RU:1}
Solvent_operation_electriciy_GL_functional_unit = {Solvent_operation_electriciy_GL:1}

Solvent_operation_heat_NG_functional_unit = {Solvent_operation_heat_NG:1}
Solvent_operation_heat_biogas_functional_unit = {Solvent_operation_heat_biogas:1}

Solvent_EoL_treat_functional_unit = {Solvent_EoL_treat:1}


# Sorbent
Sorbent_construction_functional_unit = {Sorbent_construction:1}
Sorbent_operation_material_functional_unit = {Sorbent_operation_material:1}

Sorbent_operation_electriciy_CN_functional_unit = {Sorbent_operation_electriciy_CN:1}
Sorbent_operation_electriciy_US_functional_unit = {Sorbent_operation_electriciy_US:1}
Sorbent_operation_electriciy_EU_functional_unit = {Sorbent_operation_electriciy_EU:1}
Sorbent_operation_electriciy_RU_functional_unit = {Sorbent_operation_electriciy_RU:1}
Sorbent_operation_electriciy_GL_functional_unit = {Sorbent_operation_electriciy_GL:1}

#Sorbent_operation_heat_NG_functional_unit = {Sorbent_operation_heat_NG:1}
Sorbent_operation_heat_biogas_functional_unit = {Sorbent_operation_heat_biogas:1}
#Sorbent_operation_heat_geothermal_functional_unit = {Sorbent_operation_heat_geothermal:1}
Sorbent_operation_heat_pump_CN_elec_functional_unit = {Sorbent_operation_heat_pump_CN_elec:1}
Sorbent_operation_heat_pump_US_elec_functional_unit = {Sorbent_operation_heat_pump_US_elec:1}
Sorbent_operation_heat_pump_EU_elec_functional_unit = {Sorbent_operation_heat_pump_EU_elec:1}
Sorbent_operation_heat_pump_RU_elec_functional_unit = {Sorbent_operation_heat_pump_RU_elec:1}
Sorbent_operation_heat_pump_GL_elec_functional_unit = {Sorbent_operation_heat_pump_GL_elec:1}

Sorbent_EoL_treat_functional_unit = {Sorbent_EoL_treat:1}

In [456]:
Solvent_construction_result = []
Solvent_operation_material_result = []
Solvent_operation_electriciy_CN_result = []
Solvent_operation_electriciy_US_result = []
Solvent_operation_electriciy_EU_result = []
Solvent_operation_electriciy_RU_result = []
Solvent_operation_electriciy_GL_result = []
Solvent_operation_heat_NG_result = []
Solvent_operation_heat_biogas_result = []
Solvent_EoL_treat_result = []

LCA_sol_cal_dict = collections.defaultdict(dict)

LCA_sol_cal_dict['Solvent_construction'] = {'fu' : Solvent_construction_functional_unit, 'result': Solvent_construction_result}
LCA_sol_cal_dict['Solvent_operation_material'] = {'fu' : Solvent_operation_material_functional_unit, 'result': Solvent_operation_material_result}
LCA_sol_cal_dict['Solvent_operation_electriciy_CN'] = {'fu' : Solvent_operation_electriciy_CN_functional_unit, 'result': Solvent_operation_electriciy_CN_result}
LCA_sol_cal_dict['Solvent_operation_electriciy_US'] = {'fu' : Solvent_operation_electriciy_US_functional_unit, 'result': Solvent_operation_electriciy_US_result}
LCA_sol_cal_dict['Solvent_operation_electriciy_EU'] = {'fu' : Solvent_operation_electriciy_EU_functional_unit, 'result': Solvent_operation_electriciy_EU_result}
LCA_sol_cal_dict['Solvent_operation_electriciy_RU'] = {'fu' : Solvent_operation_electriciy_RU_functional_unit, 'result': Solvent_operation_electriciy_RU_result}
LCA_sol_cal_dict['Solvent_operation_electriciy_GL'] = {'fu' : Solvent_operation_electriciy_GL_functional_unit, 'result': Solvent_operation_electriciy_GL_result}
LCA_sol_cal_dict['Solvent_operation_heat_NG'] = {'fu' : Solvent_operation_heat_NG_functional_unit, 'result': Solvent_operation_heat_NG_result}
LCA_sol_cal_dict['Solvent_operation_heat_biogas'] = {'fu' : Solvent_operation_heat_biogas_functional_unit, 'result': Solvent_operation_heat_biogas_result}
LCA_sol_cal_dict['Solvent_EoL_treat'] = {'fu' : Solvent_EoL_treat_functional_unit, 'result': Solvent_EoL_treat_result}


Sorbent_construction_result = []
Sorbent_operation_material_result = []
Sorbent_operation_electriciy_CN_result = []
Sorbent_operation_electriciy_US_result = []
Sorbent_operation_electriciy_EU_result = []
Sorbent_operation_electriciy_RU_result = []
Sorbent_operation_electriciy_GL_result = []
Sorbent_operation_heat_biogas_result = []
Sorbent_operation_heat_pump_CN_elec_result = []
Sorbent_operation_heat_pump_US_elec_result = []
Sorbent_operation_heat_pump_EU_elec_result = []
Sorbent_operation_heat_pump_RU_elec_result = []
Sorbent_operation_heat_pump_GL_elec_result = []
Sorbent_EoL_treat_result = []

LCA_sor_cal_dict = collections.defaultdict(dict)

LCA_sor_cal_dict['Sorbent_construction'] = {'fu' : Sorbent_construction_functional_unit, 'result': Sorbent_construction_result}
LCA_sor_cal_dict['Sorbent_operation_material'] = {'fu' : Sorbent_operation_material_functional_unit, 'result': Sorbent_operation_material_result}
LCA_sor_cal_dict['Sorbent_operation_electriciy_CN'] = {'fu' : Sorbent_operation_electriciy_CN_functional_unit, 'result': Sorbent_operation_electriciy_CN_result}
LCA_sor_cal_dict['Sorbent_operation_electriciy_US'] = {'fu' : Sorbent_operation_electriciy_US_functional_unit, 'result': Sorbent_operation_electriciy_US_result}
LCA_sor_cal_dict['Sorbent_operation_electriciy_EU'] = {'fu' : Sorbent_operation_electriciy_EU_functional_unit, 'result': Sorbent_operation_electriciy_EU_result}
LCA_sor_cal_dict['Sorbent_operation_electriciy_RU'] = {'fu' : Sorbent_operation_electriciy_RU_functional_unit, 'result': Sorbent_operation_electriciy_RU_result}
LCA_sor_cal_dict['Sorbent_operation_electriciy_GL'] = {'fu' : Sorbent_operation_electriciy_GL_functional_unit, 'result': Sorbent_operation_electriciy_GL_result}
LCA_sor_cal_dict['Sorbent_operation_heat_biogas'] = {'fu' : Sorbent_operation_heat_biogas_functional_unit, 'result': Sorbent_operation_heat_biogas_result}
LCA_sor_cal_dict['Sorbent_operation_heat_pump_CN_elec'] = {'fu' : Sorbent_operation_heat_pump_CN_elec_functional_unit, 'result': Sorbent_operation_heat_pump_CN_elec_result}
LCA_sor_cal_dict['Sorbent_operation_heat_pump_US_elec'] = {'fu' : Sorbent_operation_heat_pump_US_elec_functional_unit, 'result': Sorbent_operation_heat_pump_US_elec_result}
LCA_sor_cal_dict['Sorbent_operation_heat_pump_EU_elec'] = {'fu' : Sorbent_operation_heat_pump_EU_elec_functional_unit, 'result': Sorbent_operation_heat_pump_EU_elec_result}
LCA_sor_cal_dict['Sorbent_operation_heat_pump_RU_elec'] = {'fu' : Sorbent_operation_heat_pump_RU_elec_functional_unit, 'result': Sorbent_operation_heat_pump_RU_elec_result}
LCA_sor_cal_dict['Sorbent_operation_heat_pump_GL_elec'] = {'fu' : Sorbent_operation_heat_pump_GL_elec_functional_unit, 'result': Sorbent_operation_heat_pump_GL_elec_result}
LCA_sor_cal_dict['Sorbent_EoL_treat'] = {'fu' : Sorbent_EoL_treat_functional_unit, 'result': Sorbent_EoL_treat_result}


In [457]:
## Calculating the LCA result

for key in LCA_sol_cal_dict.keys():
    lca = bw.LCA(LCA_sol_cal_dict[key]['fu'])
    lca.lci()
    for method in method_key:
        lca.switch_method(method)
        lca.lcia()
        LCA_sol_cal_dict[key]['result'].append((method[2].title(), lca.score, bw.methods.get(method).get('unit')))
        
LCA_sol_table = pd.DataFrame(index=['Agricultural land occupation (square meter-year)',
                                    'Climate change (kg CO2-Eq)',
                                    'Fossil depletion (kg oil-Eq)',
                                    'Freshwater ecotoxicity (kg 14-DCB)',
                                    'Freshwater eutrophication (kg P-Eq)',
                                    'Human toxicity (kg 14-DCB)',
                                    'Ionising radiation (kg U235-Eq)', 
                                    'Marine ecotoxicity (kg 14-DCB)', 
                                    'Marine eutrophication (kg N-Eq)', 
                                    'Metal depletion (kg Fe-Eq)',                              
                                    'Natural land transformation (square meter)', 
                                    'Ozone depletion (kg CFC-11)', 
                                    'Particulate matter formation (kg PM10-Eq)',
                                    'Photochemical oxidant formation (kg NMVOC)', 
                                    'Terrestrial acidification (kg SO2-Eq)', 
                                    'Terrestrial ecotoxicity (kg 14-DCB)', 
                                    'Urban land occupation (square meter-year)', 
                                    'Water depletion (m3 water)'])

for key in LCA_sol_cal_dict.keys():
    LCA_sol_table[key] = "NA"
    for i in range(0, 18):
        LCA_sol_table[key][i] = DataFrame(LCA_sol_cal_dict[key]['result'])[[1]][1][i]
        
## Natural gas combustion for heat are at the 8th columns (Natural gas emission factor is 0.0589 kg CO2/MJ). The CO2 emission from natural gas combustion can be collected, instead of releasing to air)

LCA_sol_table.iat[1,7] = LCA_sol_table.iat[1,7] - Sol_data_input.iat[34, 3] * 0.0589

# Also need to do this calculation for biomethane scenario (biomethane emission factor is 0.0589 kg CO2/MJ), because biomethane as heat source for DAC can be considered as a NET technology. 

LCA_sol_table.iat[1,8] = LCA_sol_table.iat[1,8] - Sol_data_input.iat[34, 3] * 0.0549

# Add a new column to consider the CO2 being captured by DAC
LCA_sol_table.insert(0, 'Solvent_CO2_capture', [0, -1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], True)


#LCA_sol_table

In [458]:
for key in LCA_sor_cal_dict.keys():
    lca = bw.LCA(LCA_sor_cal_dict[key]['fu'])
    lca.lci()
    for method in method_key:
        lca.switch_method(method)
        lca.lcia()
        LCA_sor_cal_dict[key]['result'].append((method[2].title(), lca.score, bw.methods.get(method).get('unit')))
        
LCA_sor_table = pd.DataFrame(index=['Agricultural land occupation (square meter-year)',
                                    'Climate change (kg CO2-Eq)',
                                    'Fossil depletion (kg oil-Eq)',
                                    'Freshwater ecotoxicity (kg 14-DCB)',
                                    'Freshwater eutrophication (kg P-Eq)',
                                    'Human toxicity (kg 14-DCB)',
                                    'Ionising radiation (kg U235-Eq)', 
                                    'Marine ecotoxicity (kg 14-DCB)', 
                                    'Marine eutrophication (kg N-Eq)', 
                                    'Metal depletion (kg Fe-Eq)',                              
                                    'Natural land transformation (square meter)', 
                                    'Ozone depletion (kg CFC-11)', 
                                    'Particulate matter formation (kg PM10-Eq)',
                                    'Photochemical oxidant formation (kg NMVOC)', 
                                    'Terrestrial acidification (kg SO2-Eq)', 
                                    'Terrestrial ecotoxicity (kg 14-DCB)', 
                                    'Urban land occupation (square meter-year)', 
                                    'Water depletion (m3 water)'])

for key in LCA_sor_cal_dict.keys():
    LCA_sor_table[key] = "NA"
    for i in range(0, 18):
        LCA_sor_table[key][i] = DataFrame(LCA_sor_cal_dict[key]['result'])[[1]][1][i]

LCA_sor_table.insert(-0, 'Sorbent_CO2_capture', [0, -1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], True)

#LCA_sor_table
    


In [348]:
## Create the excel and write the first sheet (using the LCA result calculated based on ecoinvent_2020 database)
## Depending on the scenario (BAU, RCP19_DAC, RCP19_DAC_LR, RCP19_DAC_LR_low, RCP19_DAC_LR_high), we write the results 
## into diffeernt excel spreadsheet

## "BAU" represents the results calculated under SSP2-Baseline scenario
sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_BAU.xlsx'
sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_BAU.xlsx'


## "RCP19_DAC" represents the results calculated under SSP2-RCP19 w/ DAC (no learning) scenario
#sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC.xlsx'
#sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC.xlsx'


## "RCP19_DAC_LR" represents the results calculated under SSP2-RCP19 w/ DAC (with learning) scenario
#sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR.xlsx'
#sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR.xlsx'

## "RCP19_DAC_LR_low" represents the results calculated under SSP2-RCP19 w/ DAC (with slow learning rates) scenario
#sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR_low.xlsx'
#sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR_low.xlsx'

## "RCP19_DAC_LR_high" represents the results calculated under SSP2-RCP19 w/ DAC (with fast learning rates) scenario
#sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR_high.xlsx'
#sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR_high.xlsx'

writer = pd.ExcelWriter(sol_path, engine = 'xlsxwriter')
LCA_sol_table.to_excel(writer, sheet_name = database_name[10:])

writer.save()
writer.close()

writer = pd.ExcelWriter(sor_path, engine = 'xlsxwriter')
LCA_sor_table.to_excel(writer, sheet_name = database_name[10:])

writer.save()
writer.close()

C:\Users\yqiu\Anaconda3\envs\DAC_LCA_project\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [459]:
## Write the following excel sheet (for the LCA result calculated for following year (2030-2100))

sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_BAU.xlsx'
sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_BAU.xlsx'

#sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC.xlsx'
#sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC.xlsx'

#sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR.xlsx'
#sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR.xlsx'

#sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR_low.xlsx'
#sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR_low.xlsx'

#sol_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR_high.xlsx'
#sor_path = r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR_high.xlsx'

book = load_workbook(sol_path)
writer = pd.ExcelWriter(sol_path, engine = 'openpyxl')
writer.book = book
LCA_sol_table.to_excel(writer, sheet_name = database_name[10:])
writer.save()
writer.close()

book = load_workbook(sor_path)
writer = pd.ExcelWriter(sor_path, engine = 'openpyxl')
writer.book = book
LCA_sor_table.to_excel(writer, sheet_name = database_name[10:])
writer.save()
writer.close()

## 2. Summarize the LCA results (across different years and technologies) into a table 
##### After calculating the LCA results for each year and scenarios, we summarize the LCA results calculated in the previous step into a overall table. 

In [460]:
## To calculate the overall results

LCA_sol_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_BAU.xlsx')
#LCA_sol_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC.xlsx')
#LCA_sol_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR.xlsx')
#LCA_sol_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR_low.xlsx')
#LCA_sol_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR_high.xlsx')

LCA_sol_dict = {}

for key in LCA_sol_table_name.sheet_names:
    LCA_sol_dict[key] = pd.read_excel(LCA_sol_table_name, sheet_name = key)

#print(LCA_sol_dict)

LCA_sol_country_dict = {}
for key in LCA_sol_dict.keys():
    tem = LCA_sol_dict[key]
    tem['Solvent_NG_CN'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_CN'] + tem['Solvent_operation_heat_NG'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    tem['Solvent_NG_US'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_US'] + tem['Solvent_operation_heat_NG'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    tem['Solvent_NG_EU'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_EU'] + tem['Solvent_operation_heat_NG'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    tem['Solvent_NG_RU'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_RU'] + tem['Solvent_operation_heat_NG'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    tem['Solvent_NG_GL'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_GL'] + tem['Solvent_operation_heat_NG'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    
    tem['Solvent_BG_CN'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_CN'] + tem['Solvent_operation_heat_biogas'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    tem['Solvent_BG_US'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_US'] + tem['Solvent_operation_heat_biogas'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    tem['Solvent_BG_EU'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_EU'] + tem['Solvent_operation_heat_biogas'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    tem['Solvent_BG_RU'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_RU'] + tem['Solvent_operation_heat_biogas'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    tem['Solvent_BG_GL'] = tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_GL'] + tem['Solvent_operation_heat_biogas'] + tem['Solvent_EoL_treat'] + tem['Solvent_CO2_capture']
    #tem['year'] = key[0:4]
    
    tem_country_total = tem[['Unnamed: 0', 'Solvent_NG_CN', 'Solvent_NG_US', 'Solvent_NG_EU', 'Solvent_NG_RU', 'Solvent_NG_GL', 'Solvent_BG_CN', 'Solvent_BG_US', 'Solvent_BG_EU', 'Solvent_BG_RU', 'Solvent_BG_GL']].copy()
    tem_country_total = tem_country_total.set_index(['Unnamed: 0'])
    LCA_sol_country_dict[key[0:4]] = tem_country_total
    
#print(LCA_sol_country_dict)

LCA_sol_country_summary_table = pd.concat(LCA_sol_country_dict ,axis=0)
LCA_sol_country_summary_table.index.names = ["Year", 'Impact category']
#LCA_sol_country_summary_table = LCA_sol_country_summary_table.reset_index(['Year'])
LCA_sol_country_summary_table

Solvent_NG_CN  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)      13.859927   
     Climate change (kg CO2-Eq)                          -562.961221   
     Fossil depletion (kg oil-Eq)                         260.830716   
     Freshwater ecotoxicity (kg 14-DCB)                     7.772110   
     Freshwater eutrophication (kg P-Eq)                    0.108209   
...                                                              ...   
2100 Photochemical oxidant formation (kg NMVOC)             0.441947   
     Terrestrial acidification (kg SO2-Eq)                  0.398124   
     Terrestrial ecotoxicity (kg 14-DCB)                    0.047695   
     Urban land occupation (square meter-year)              5.829161   
     Water depletion (m3 water)                             6.175353   

                                                       Solvent_NG_US  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)      25.211339   
     Climate change (kg CO2-Eq)                          -662.779794   
     Fossil depletion (kg oil-Eq)                         254.077664   
     Freshwater ecotoxicity (kg 14-DCB)                     9.946907   
     Freshwater eutrophication (kg P-Eq)                    0.259797   
...                                                              ...   
2100 Photochemical oxidant formation (kg NMVOC)             0.445220   
     Terrestrial acidification (kg SO2-Eq)                  0.397401   
     Terrestrial ecotoxicity (kg 14-DCB)                    0.044053   
     Urban land occupation (square meter-year)              4.587791   
     Water depletion (m3 water)                             6.143234   

                                                       Solvent_NG_EU  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)      51.144464   
     Climate change (kg CO2-Eq)                          -736.258819   
     Fossil depletion (kg oil-Eq)                         231.364247   
     Freshwater ecotoxicity (kg 14-DCB)                     9.093438   
     Freshwater eutrophication (kg P-Eq)                    0.186342   
...                                                              ...   
2100 Photochemical oxidant formation (kg NMVOC)             0.463029   
     Terrestrial acidification (kg SO2-Eq)                  0.415589   
     Terrestrial ecotoxicity (kg 14-DCB)                    0.047553   
     Urban land occupation (square meter-year)              4.525093   
     Water depletion (m3 water)                             6.236919   

                                                       Solvent_NG_RU  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)       4.972276   
     Climate change (kg CO2-Eq)                          -589.867483   
     Fossil depletion (kg oil-Eq)                         289.390178   
     Freshwater ecotoxicity (kg 14-DCB)                     8.586663   
     Freshwater eutrophication (kg P-Eq)                    0.232910   
...                                                              ...   
2100 Photochemical oxidant formation (kg NMVOC)             0.520834   
     Terrestrial acidification (kg SO2-Eq)                  0.461899   
     Terrestrial ecotoxicity (kg 14-DCB)                    0.036608   
     Urban land occupation (square meter-year)              3.398288   
     Water depletion (m3 water)                             6.474398   

                                                       Solvent_NG_GL  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)      23.801765   
     Climate change (kg CO2-Eq)                          -631.829541   
     Fossil depleti

In [461]:
LCA_sor_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_BAU.xlsx')
#LCA_sor_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC.xlsx')
#LCA_sor_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR.xlsx')
#LCA_sor_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR_low.xlsx')
#LCA_sor_table_name = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR_high.xlsx')

LCA_sor_dict = {}

for key in LCA_sor_table_name.sheet_names:
    LCA_sor_dict[key] = pd.read_excel(LCA_sor_table_name, sheet_name = key)

#print(LCA_sor_dict)

LCA_sor_country_dict = {}
for key in LCA_sor_dict.keys():
    tem = LCA_sor_dict[key]
#    tem['Sorbent_NG_CN']              = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_CN'] + tem['Sorbent_operation_heat_NG']
#    tem['Sorbent_NG_US']                 = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_US'] + tem['Sorbent_operation_heat_NG']
#    tem['Sorbent_NG_EU']         = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_FR'] + tem['Sorbent_operation_heat_NG']
    
    tem['Sorbent_HP_CN']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_CN'] + tem['Sorbent_operation_heat_pump_CN_elec'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    tem['Sorbent_HP_US']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_US'] + tem['Sorbent_operation_heat_pump_US_elec'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    tem['Sorbent_HP_EU']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_EU'] + tem['Sorbent_operation_heat_pump_EU_elec'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    tem['Sorbent_HP_RU']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_RU'] + tem['Sorbent_operation_heat_pump_RU_elec'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    tem['Sorbent_HP_GL']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_GL'] + tem['Sorbent_operation_heat_pump_GL_elec'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    
    tem['Sorbent_BG_CN']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_CN'] + tem['Sorbent_operation_heat_biogas'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    tem['Sorbent_BG_US']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_US'] + tem['Sorbent_operation_heat_biogas'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    tem['Sorbent_BG_EU']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_EU'] + tem['Sorbent_operation_heat_biogas'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    tem['Sorbent_BG_RU']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_RU'] + tem['Sorbent_operation_heat_biogas'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    tem['Sorbent_BG_GL']  = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_GL'] + tem['Sorbent_operation_heat_biogas'] + tem['Sorbent_EoL_treat'] + tem['Sorbent_CO2_capture']
    
#    tem['Sorbent_GT_CN']      = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_CN'] + tem['Sorbent_operation_heat_geothermal']
#    tem['Sorbent_GT_US']         = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_US'] + tem['Sorbent_operation_heat_geothermal']
#    tem['Sorbent_GT_EU'] = tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_FR'] + tem['Sorbent_operation_heat_geothermal']
    #tem['year'] = key[0:4]
    
    tem_country_total = tem[['Unnamed: 0', 'Sorbent_HP_CN', 'Sorbent_HP_US', 'Sorbent_HP_EU', 'Sorbent_HP_RU', 'Sorbent_HP_GL', 'Sorbent_BG_CN', 'Sorbent_BG_US', 'Sorbent_BG_EU', 'Sorbent_BG_RU', 'Sorbent_BG_GL']].copy()
    tem_country_total = tem_country_total.set_index(['Unnamed: 0'])
    LCA_sor_country_dict[key[0:4]] = tem_country_total
    
#print(LCA_sor_country_dict)

LCA_sor_country_summary_table = pd.concat(LCA_sor_country_dict ,axis=0)
LCA_sor_country_summary_table.index.names = ["Year", 'Impact category']
#LCA_sor_country_summary_table = LCA_sor_country_summary_table.reset_index(['Year'])
LCA_sor_country_summary_table

Sorbent_HP_CN  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)      29.896650   
     Climate change (kg CO2-Eq)                          -118.940922   
     Fossil depletion (kg oil-Eq)                         216.124916   
     Freshwater ecotoxicity (kg 14-DCB)                    16.498642   
     Freshwater eutrophication (kg P-Eq)                    0.237735   
...                                                              ...   
2100 Photochemical oxidant formation (kg NMVOC)             0.224265   
     Terrestrial acidification (kg SO2-Eq)                  0.254305   
     Terrestrial ecotoxicity (kg 14-DCB)                    0.054600   
     Urban land occupation (square meter-year)             10.687195   
     Water depletion (m3 water)                             0.553528   

                                                       Sorbent_HP_US  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)      57.195227   
     Climate change (kg CO2-Eq)                          -358.990795   
     Fossil depletion (kg oil-Eq)                         199.884758   
     Freshwater ecotoxicity (kg 14-DCB)                    21.728727   
     Freshwater eutrophication (kg P-Eq)                    0.602284   
...                                                              ...   
2100 Photochemical oxidant formation (kg NMVOC)             0.230929   
     Terrestrial acidification (kg SO2-Eq)                  0.252833   
     Terrestrial ecotoxicity (kg 14-DCB)                    0.047182   
     Urban land occupation (square meter-year)              8.159386   
     Water depletion (m3 water)                             0.488123   

                                                       Sorbent_HP_EU  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)     119.560807   
     Climate change (kg CO2-Eq)                          -535.697692   
     Fossil depletion (kg oil-Eq)                         145.262130   
     Freshwater ecotoxicity (kg 14-DCB)                    19.676254   
     Freshwater eutrophication (kg P-Eq)                    0.425634   
...                                                              ...   
2100 Photochemical oxidant formation (kg NMVOC)             0.267195   
     Terrestrial acidification (kg SO2-Eq)                  0.289869   
     Terrestrial ecotoxicity (kg 14-DCB)                    0.054310   
     Urban land occupation (square meter-year)              8.031713   
     Water depletion (m3 water)                             0.678896   

                                                       Sorbent_HP_RU  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)       8.523077   
     Climate change (kg CO2-Eq)                          -183.646765   
     Fossil depletion (kg oil-Eq)                         284.806473   
     Freshwater ecotoxicity (kg 14-DCB)                    18.457529   
     Freshwater eutrophication (kg P-Eq)                    0.537623   
...                                                              ...   
2100 Photochemical oxidant formation (kg NMVOC)             0.384903   
     Terrestrial acidification (kg SO2-Eq)                  0.384170   
     Terrestrial ecotoxicity (kg 14-DCB)                    0.032024   
     Urban land occupation (square meter-year)              5.737191   
     Water depletion (m3 water)                             1.162477   

                                                       Sorbent_HP_GL  \
Year Impact category                                                   
2020 Agricultural land occupation (square meter-year)      53.805397   
     Climate change (kg CO2-Eq)                          -284.559714   
     Fossil depleti

In [462]:
LCA_sol_sor_country_summary_table = pd.concat([LCA_sol_country_summary_table, LCA_sor_country_summary_table], axis=1)
LCA_sol_sor_country_summary_table = LCA_sol_sor_country_summary_table.reset_index()

LCA_sol_sor_country_summary_table_gather = LCA_sol_sor_country_summary_table.melt(id_vars=['Impact category', 'Year'], 
                                                                                  value_vars=['Solvent_NG_CN', 
                                                                                              'Solvent_NG_US',
                                                                                              'Solvent_NG_EU',
                                                                                              'Solvent_NG_RU',
                                                                                              'Solvent_NG_GL',
                                                                                              'Solvent_BG_CN', 
                                                                                              'Solvent_BG_US',
                                                                                              'Solvent_BG_EU',
                                                                                              'Solvent_BG_RU',
                                                                                              'Solvent_BG_GL',
                                                                                            #  'Sorbent_NG_China',
                                                                                            #  'Sorbent_NG_US',
                                                                                            #  'Sorbent_NG_Western_EU',
                                                                                              'Sorbent_HP_CN',
                                                                                              'Sorbent_HP_US',
                                                                                              'Sorbent_HP_EU',
                                                                                              'Sorbent_HP_RU',
                                                                                              'Sorbent_HP_GL',
                                                                                              'Sorbent_BG_CN',
                                                                                              'Sorbent_BG_US',
                                                                                              'Sorbent_BG_EU',
                                                                                              'Sorbent_BG_RU',
                                                                                              'Sorbent_BG_GL'],
                                                                                  var_name = 'Heat_elec_region', 
                                                                                  value_name = 'value')

LCA_sol_sor_country_summary_table_gather = LCA_sol_sor_country_summary_table_gather.set_index('Impact category')
LCA_sol_sor_country_summary_table_gather
LCA_sol_sor_country_summary_table_spread = LCA_sol_sor_country_summary_table_gather.pivot_table(index=['Impact category', 'Heat_elec_region'],columns='Year',values='value',fill_value=0)
LCA_sol_sor_country_summary_table_spread = LCA_sol_sor_country_summary_table_spread.reset_index()


## Combining the final LCA results into an over table (including all DACCS technologies, years)
LCA_sol_sor_country_summary_table_spread.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_sor_country_summary_table_spread_BAU.xlsx', index = False) 
#LCA_sol_sor_country_summary_table_spread.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_sor_country_summary_table_spread_RCP19_DAC.xlsx', index = False) 
#LCA_sol_sor_country_summary_table_spread.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_sor_country_summary_table_spread_RCP19_DAC_LR.xlsx', index = False) 
#LCA_sol_sor_country_summary_table_spread.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_sor_country_summary_table_spread_RCP19_DAC_LR_low.xlsx', index = False) 
#LCA_sol_sor_country_summary_table_spread.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_sor_country_summary_table_spread_RCP19_DAC_LR_high.xlsx', index = False) 

## 3. Contribution Analysis
##### Showing the contribution of each life cycle stages to the total amount of impact. We only calculate the results for SSP2-RCP1.9 w/DACCS scenario with and without learning. For the with learning case, we use the results under reference learning rates. The results are presented in SI-Figure 6

In [ ]:
## Import the LCA results from previous steps

LCA_sol_RCP19_DAC_con = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC.xlsx')
LCA_sol_RCP19_DAC_LR_con = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_table_RCP19_DAC_LR.xlsx')


LCA_sor_RCP19_DAC_con = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC.xlsx')
LCA_sor_RCP19_DAC_LR_con = pd.ExcelFile(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_table_RCP19_DAC_LR.xlsx')


In [ ]:
## Calculate the overall results for solvent-based DACCS (SSP2-RCP1.9 w/DAC no learning)

LCA_sol_RCP19_DAC_con_dict = {}

for key in LCA_sol_RCP19_DAC_con.sheet_names:
    LCA_sol_RCP19_DAC_con_dict[key] = pd.read_excel(LCA_sol_RCP19_DAC_con, sheet_name = key)

LCA_sol_RCP19_DAC_con_country_dict = {}
for key in LCA_sol_RCP19_DAC_con_dict.keys():
    tem = LCA_sol_RCP19_DAC_con_dict[key]
    tem['Solvent_NG_US'] = tem['Solvent_CO2_capture'] + tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_US'] + tem['Solvent_operation_heat_NG'] + tem['Solvent_EoL_treat']
    tem['Solvent_BG_US'] = tem['Solvent_CO2_capture'] + tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_US'] + tem['Solvent_operation_heat_biogas'] + tem['Solvent_EoL_treat']
   
    tem['Solvent_CO2_capture_NG_US'] = tem['Solvent_CO2_capture']
    tem['Solvent_construction_NG_US'] = tem['Solvent_construction']
    tem['Solvent_operation_material_NG_US'] = tem['Solvent_operation_material']
    tem['Solvent_operation_electriciy_NG_US'] = tem['Solvent_operation_electriciy_US']
    tem['Solvent_operation_heat_NG_US'] = tem['Solvent_operation_heat_NG']
    tem['Solvent_EoL_treat_NG_US'] = tem['Solvent_EoL_treat']
  
    
    tem['Solvent_CO2_capture_BG_US'] = tem['Solvent_CO2_capture']
    tem['Solvent_construction_BG_US'] = tem['Solvent_construction']
    tem['Solvent_operation_material_BG_US'] = tem['Solvent_operation_material']
    tem['Solvent_operation_electriciy_BG_US'] = tem['Solvent_operation_electriciy_US']
    tem['Solvent_operation_heat_BG_US'] = tem['Solvent_operation_heat_biogas']
    tem['Solvent_EoL_treat_BG_US'] = tem['Solvent_EoL_treat']
    

    
    tem_country_total = tem[['Unnamed: 0', 
                             'Solvent_CO2_capture_NG_US', 'Solvent_construction_NG_US', 'Solvent_operation_material_NG_US', 'Solvent_operation_electriciy_NG_US', 'Solvent_operation_heat_NG_US', 'Solvent_EoL_treat_NG_US',
                             'Solvent_CO2_capture_BG_US', 'Solvent_construction_BG_US', 'Solvent_operation_material_BG_US', 'Solvent_operation_electriciy_BG_US', 'Solvent_operation_heat_BG_US', 'Solvent_EoL_treat_BG_US']].copy()
   
    tem_country_total = tem_country_total.set_index(['Unnamed: 0'])
    LCA_sol_RCP19_DAC_con_country_dict[key[0:4]] = tem_country_total

#LCA_sol_RCP19_con_country_dict

LCA_sol_contribution_summary_table = pd.concat(LCA_sol_RCP19_DAC_con_country_dict ,axis=0)
LCA_sol_contribution_summary_table.index.names = ["Year", 'Impact category']

#LCA_sol_contribution_summary_table

# NG_US
LCA_sol_contribution_NG_US_table = LCA_sol_contribution_summary_table[['Solvent_CO2_capture_NG_US', 'Solvent_construction_NG_US', 'Solvent_operation_material_NG_US', 'Solvent_operation_electriciy_NG_US', 'Solvent_operation_heat_NG_US', 'Solvent_EoL_treat_NG_US']]
LCA_sol_contribution_NG_US_table = LCA_sol_contribution_NG_US_table.reset_index()
LCA_sol_contribution_NG_US_table.columns=['Year','Impact_category', '0_CO2_capture', '1_Construction','2_Op_material','3_Op_electricity', '4_Op_heat', '5_EoL_treat']
LCA_sol_contribution_NG_US_table_gather = LCA_sol_contribution_NG_US_table.melt(id_vars=['Year', 'Impact_category'], 
                                                                                value_vars=['0_CO2_capture', '1_Construction', '2_Op_material', '3_Op_electricity', '4_Op_heat', '5_EoL_treat'],
                                                                                var_name = 'process', 
                                                                                value_name = 'value')
LCA_sol_contribution_NG_US_table_gather['Tech_heat_region'] = 'Sol_NG_US'
LCA_sol_contribution_NG_US_table_gather = LCA_sol_contribution_NG_US_table_gather[list(('Year', 'Impact_category', 'Tech_heat_region', 'process', 'value'))]
LCA_sol_contribution_NG_US_table_gather = LCA_sol_contribution_NG_US_table_gather.sort_values(by=['Year', 'Impact_category', 'Tech_heat_region', 'process'])
#LCA_sol_contribution_NG_US_table_gather


# BG_US
LCA_sol_contribution_BG_US_table = LCA_sol_contribution_summary_table[['Solvent_CO2_capture_BG_US', 'Solvent_construction_BG_US', 'Solvent_operation_material_BG_US', 'Solvent_operation_electriciy_BG_US', 'Solvent_operation_heat_BG_US', 'Solvent_EoL_treat_BG_US']]
LCA_sol_contribution_BG_US_table = LCA_sol_contribution_BG_US_table.reset_index()
LCA_sol_contribution_BG_US_table.columns=['Year','Impact_category','0_CO2_capture', '1_Construction','2_Op_material','3_Op_electricity', '4_Op_heat', '5_EoL_treat']
LCA_sol_contribution_BG_US_table_gather = LCA_sol_contribution_BG_US_table.melt(id_vars=['Year', 'Impact_category'], 
                                                                                value_vars=['0_CO2_capture', '1_Construction', '2_Op_material', '3_Op_electricity', '4_Op_heat', '5_EoL_treat'],
                                                                                var_name = 'process', 
                                                                                value_name = 'value')
LCA_sol_contribution_BG_US_table_gather['Tech_heat_region'] = 'Sol_BG_US'
LCA_sol_contribution_BG_US_table_gather = LCA_sol_contribution_BG_US_table_gather[list(('Year', 'Impact_category', 'Tech_heat_region', 'process', 'value'))]
LCA_sol_contribution_BG_US_table_gather = LCA_sol_contribution_BG_US_table_gather.sort_values(by=['Year', 'Impact_category', 'Tech_heat_region', 'process'])
#LCA_sol_contribution_BG_US_table_gather


LCA_sol_RCP19_DAC_contribution_all_summary_table = pd.concat([LCA_sol_contribution_NG_US_table_gather,
                                                    LCA_sol_contribution_BG_US_table_gather], axis=0)
LCA_sol_RCP19_DAC_contribution_all_summary_table

In [ ]:
## Calculate the overall results for solvent-based DACCS (SSP2-RCP1.9 w/DAC with learning)

LCA_sol_RCP19_DAC_LR_con_dict = {}

for key in LCA_sol_RCP19_DAC_LR_con.sheet_names:
    LCA_sol_RCP19_DAC_LR_con_dict[key] = pd.read_excel(LCA_sol_RCP19_DAC_LR_con, sheet_name = key)

LCA_sol_RCP19_DAC_LR_country_dict = {}
for key in LCA_sol_RCP19_DAC_LR_con_dict.keys():
    tem = LCA_sol_RCP19_DAC_LR_con_dict[key]
    tem['Solvent_NG_US'] = tem['Solvent_CO2_capture'] + tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_US'] + tem['Solvent_operation_heat_NG'] + tem['Solvent_EoL_treat']
    tem['Solvent_BG_US'] = tem['Solvent_CO2_capture'] + tem['Solvent_construction'] + tem['Solvent_operation_material'] + tem['Solvent_operation_electriciy_US'] + tem['Solvent_operation_heat_biogas'] + tem['Solvent_EoL_treat']
   
    tem['Solvent_CO2_capture_NG_US'] = tem['Solvent_CO2_capture']
    tem['Solvent_construction_NG_US'] = tem['Solvent_construction']
    tem['Solvent_operation_material_NG_US'] = tem['Solvent_operation_material']
    tem['Solvent_operation_electriciy_NG_US'] = tem['Solvent_operation_electriciy_US']
    tem['Solvent_operation_heat_NG_US'] = tem['Solvent_operation_heat_NG']
    tem['Solvent_EoL_treat_NG_US'] = tem['Solvent_EoL_treat']
  
    
    tem['Solvent_CO2_capture_BG_US'] = tem['Solvent_CO2_capture']
    tem['Solvent_construction_BG_US'] = tem['Solvent_construction']
    tem['Solvent_operation_material_BG_US'] = tem['Solvent_operation_material']
    tem['Solvent_operation_electriciy_BG_US'] = tem['Solvent_operation_electriciy_US']
    tem['Solvent_operation_heat_BG_US'] = tem['Solvent_operation_heat_biogas']
    tem['Solvent_EoL_treat_BG_US'] = tem['Solvent_EoL_treat']
    

    
    tem_country_total = tem[['Unnamed: 0', 
                             'Solvent_CO2_capture_NG_US', 'Solvent_construction_NG_US', 'Solvent_operation_material_NG_US', 'Solvent_operation_electriciy_NG_US', 'Solvent_operation_heat_NG_US', 'Solvent_EoL_treat_NG_US',
                             'Solvent_CO2_capture_BG_US', 'Solvent_construction_BG_US', 'Solvent_operation_material_BG_US', 'Solvent_operation_electriciy_BG_US', 'Solvent_operation_heat_BG_US', 'Solvent_EoL_treat_BG_US']].copy()
   
    tem_country_total = tem_country_total.set_index(['Unnamed: 0'])
    LCA_sol_RCP19_DAC_LR_country_dict[key[0:4]] = tem_country_total

#LCA_sol_RCP19_DAC_country_dict

LCA_sol_contribution_summary_table = pd.concat(LCA_sol_RCP19_DAC_LR_country_dict ,axis=0)
LCA_sol_contribution_summary_table.index.names = ["Year", 'Impact category']

#LCA_sol_contribution_summary_table

# NG_US
LCA_sol_contribution_NG_US_table = LCA_sol_contribution_summary_table[['Solvent_CO2_capture_NG_US', 'Solvent_construction_NG_US', 'Solvent_operation_material_NG_US', 'Solvent_operation_electriciy_NG_US', 'Solvent_operation_heat_NG_US', 'Solvent_EoL_treat_NG_US']]
LCA_sol_contribution_NG_US_table = LCA_sol_contribution_NG_US_table.reset_index()
LCA_sol_contribution_NG_US_table.columns=['Year','Impact_category','0_CO2_capture', '1_Construction','2_Op_material','3_Op_electricity', '4_Op_heat', '5_EoL_treat']
LCA_sol_contribution_NG_US_table_gather = LCA_sol_contribution_NG_US_table.melt(id_vars=['Year', 'Impact_category'], 
                                                                                value_vars=['0_CO2_capture', '1_Construction', '2_Op_material', '3_Op_electricity', '4_Op_heat', '5_EoL_treat'],
                                                                                var_name = 'process', 
                                                                                value_name = 'value')
LCA_sol_contribution_NG_US_table_gather['Tech_heat_region'] = 'Sol_NG_US'
LCA_sol_contribution_NG_US_table_gather = LCA_sol_contribution_NG_US_table_gather[list(('Year', 'Impact_category', 'Tech_heat_region', 'process', 'value'))]
LCA_sol_contribution_NG_US_table_gather = LCA_sol_contribution_NG_US_table_gather.sort_values(by=['Year', 'Impact_category', 'Tech_heat_region', 'process'])
#LCA_sol_contribution_NG_US_table_gather


# BG_US
LCA_sol_contribution_BG_US_table = LCA_sol_contribution_summary_table[['Solvent_CO2_capture_BG_US', 'Solvent_construction_BG_US', 'Solvent_operation_material_BG_US', 'Solvent_operation_electriciy_BG_US', 'Solvent_operation_heat_BG_US', 'Solvent_EoL_treat_BG_US']]
LCA_sol_contribution_BG_US_table = LCA_sol_contribution_BG_US_table.reset_index()
LCA_sol_contribution_BG_US_table.columns=['Year','Impact_category','0_CO2_capture', '1_Construction','2_Op_material','3_Op_electricity', '4_Op_heat', '5_EoL_treat']
LCA_sol_contribution_BG_US_table_gather = LCA_sol_contribution_BG_US_table.melt(id_vars=['Year', 'Impact_category'], 
                                                                                value_vars=['0_CO2_capture', '1_Construction', '2_Op_material', '3_Op_electricity', '4_Op_heat', '5_EoL_treat'],
                                                                                var_name = 'process', 
                                                                                value_name = 'value')
LCA_sol_contribution_BG_US_table_gather['Tech_heat_region'] = 'Sol_BG_US'
LCA_sol_contribution_BG_US_table_gather = LCA_sol_contribution_BG_US_table_gather[list(('Year', 'Impact_category', 'Tech_heat_region', 'process', 'value'))]
LCA_sol_contribution_BG_US_table_gather = LCA_sol_contribution_BG_US_table_gather.sort_values(by=['Year', 'Impact_category', 'Tech_heat_region', 'process'])
#LCA_sol_contribution_BG_US_table_gather


LCA_sol_RCP19_DAC_LR_contribution_all_summary_table = pd.concat([LCA_sol_contribution_NG_US_table_gather,
                                                    LCA_sol_contribution_BG_US_table_gather], axis=0)
LCA_sol_RCP19_DAC_LR_contribution_all_summary_table

In [ ]:
## Calculate the overall results for sorbent-based DACCS (SSP2-RCP1.9 w/DAC no learning)

LCA_sor_RCP19_DAC_con_dict = {}

for key in LCA_sor_RCP19_DAC_con.sheet_names:
    LCA_sor_RCP19_DAC_con_dict[key] = pd.read_excel(LCA_sor_RCP19_DAC_con, sheet_name = key)



LCA_sor_RCP19_DAC_country_dict = {}
for key in LCA_sor_RCP19_DAC_con_dict.keys():
    tem = LCA_sor_RCP19_DAC_con_dict[key]
    tem['Sorbent_HP_US'] = tem['Sorbent_CO2_capture'] + tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_US'] + tem['Sorbent_operation_heat_pump_US_elec'] + tem['Sorbent_EoL_treat']
    tem['Sorbent_BG_US'] = tem['Sorbent_CO2_capture'] + tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_US'] + tem['Sorbent_operation_heat_biogas'] + tem['Sorbent_EoL_treat']

    tem['Sorbent_CO2_capture_HP_US'] = tem['Sorbent_CO2_capture']
    tem['Sorbent_construction_HP_US'] = tem['Sorbent_construction']
    tem['Sorbent_operation_material_HP_US'] = tem['Sorbent_operation_material']
    tem['Sorbent_operation_electriciy_HP_US'] = tem['Sorbent_operation_electriciy_US']
    tem['Sorbent_operation_heat_HP_US'] = tem['Sorbent_operation_heat_pump_US_elec']
    tem['Sorbent_EoL_treat_HP_US'] = tem['Sorbent_EoL_treat']
    
    tem['Sorbent_CO2_capture_BG_US'] = tem['Sorbent_CO2_capture']
    tem['Sorbent_construction_BG_US'] = tem['Sorbent_construction']
    tem['Sorbent_operation_material_BG_US'] = tem['Sorbent_operation_material']
    tem['Sorbent_operation_electriciy_BG_US'] = tem['Sorbent_operation_electriciy_US']
    tem['Sorbent_operation_heat_BG_US'] = tem['Sorbent_operation_heat_biogas']
    tem['Sorbent_EoL_treat_BG_US'] = tem['Sorbent_EoL_treat']
    
    tem_country_total = tem[['Unnamed: 0', 
                             'Sorbent_CO2_capture_HP_US', 'Sorbent_construction_HP_US', 'Sorbent_operation_material_HP_US', 'Sorbent_operation_electriciy_HP_US', 'Sorbent_operation_heat_HP_US', 'Sorbent_EoL_treat_HP_US',  
                             'Sorbent_CO2_capture_BG_US', 'Sorbent_construction_BG_US', 'Sorbent_operation_material_BG_US', 'Sorbent_operation_electriciy_BG_US', 'Sorbent_operation_heat_BG_US', 'Sorbent_EoL_treat_BG_US']].copy()
    tem_country_total = tem_country_total.set_index(['Unnamed: 0'])
    LCA_sor_RCP19_DAC_country_dict[key[0:4]] = tem_country_total

LCA_sor_RCP19_country_dict
LCA_sor_contribution_summary_table = pd.concat(LCA_sor_RCP19_DAC_country_dict ,axis=0)
LCA_sor_contribution_summary_table.index.names = ["Year", 'Impact category']


# HP_US
LCA_sor_contribution_HP_US_table = LCA_sor_contribution_summary_table[['Sorbent_CO2_capture_HP_US', 'Sorbent_construction_HP_US', 'Sorbent_operation_material_HP_US', 'Sorbent_operation_electriciy_HP_US', 'Sorbent_operation_heat_HP_US', 'Sorbent_EoL_treat_HP_US']]
LCA_sor_contribution_HP_US_table = LCA_sor_contribution_HP_US_table.reset_index()
LCA_sor_contribution_HP_US_table.columns=['Year','Impact_category','0_CO2_capture', '1_Construction','2_Op_material','3_Op_electricity', '4_Op_heat', '5_EoL_treat']
LCA_sor_contribution_HP_US_table_gather = LCA_sor_contribution_HP_US_table.melt(id_vars=['Year', 'Impact_category'], 
                                                                                value_vars=['0_CO2_capture', '1_Construction', '2_Op_material', '3_Op_electricity', '4_Op_heat', '5_EoL_treat'],
                                                                                var_name = 'process', 
                                                                                value_name = 'value')
LCA_sor_contribution_HP_US_table_gather['Tech_heat_region'] = 'Sor_HP_US'
LCA_sor_contribution_HP_US_table_gather = LCA_sor_contribution_HP_US_table_gather[list(('Year', 'Impact_category', 'Tech_heat_region', 'process', 'value'))]
LCA_sor_contribution_HP_US_table_gather = LCA_sor_contribution_HP_US_table_gather.sort_values(by=['Year', 'Impact_category', 'Tech_heat_region', 'process'])
#LCA_sor_contribution_HP_US_table_gather


# BG_US
LCA_sor_contribution_BG_US_table = LCA_sor_contribution_summary_table[['Sorbent_CO2_capture_BG_US', 'Sorbent_construction_BG_US', 'Sorbent_operation_material_BG_US', 'Sorbent_operation_electriciy_BG_US', 'Sorbent_operation_heat_BG_US', 'Sorbent_EoL_treat_BG_US']]
LCA_sor_contribution_BG_US_table = LCA_sor_contribution_BG_US_table.reset_index()
LCA_sor_contribution_BG_US_table.columns=['Year','Impact_category','0_CO2_capture', '1_Construction','2_Op_material','3_Op_electricity', '4_Op_heat', '5_EoL_treat']
LCA_sor_contribution_BG_US_table_gather = LCA_sor_contribution_BG_US_table.melt(id_vars=['Year', 'Impact_category'], 
                                                                                value_vars=['0_CO2_capture', '1_Construction', '2_Op_material', '3_Op_electricity', '4_Op_heat', '5_EoL_treat'],
                                                                                var_name = 'process', 
                                                                                value_name = 'value')
LCA_sor_contribution_BG_US_table_gather['Tech_heat_region'] = 'Sor_BG_US'
LCA_sor_contribution_BG_US_table_gather = LCA_sor_contribution_BG_US_table_gather[list(('Year', 'Impact_category', 'Tech_heat_region', 'process', 'value'))]
LCA_sor_contribution_BG_US_table_gather = LCA_sor_contribution_BG_US_table_gather.sort_values(by=['Year', 'Impact_category', 'Tech_heat_region', 'process'])
#LCA_sor_contribution_BG_US_table_gather


LCA_sor_RCP19_DAC_contribution_all_summary_table = pd.concat([LCA_sor_contribution_HP_US_table_gather,
                                                LCA_sor_contribution_BG_US_table_gather], axis=0)
LCA_sor_RCP19_DAC_contribution_all_summary_table

In [ ]:
## Calculate the overall results for sorbent-based DACCS (SSP2-RCP1.9 w/DAC with learning)

LCA_sor_RCP19_DAC_LR_con_dict = {}

for key in LCA_sor_RCP19_DAC_LR_con.sheet_names:
    LCA_sor_RCP19_DAC_LR_con_dict[key] = pd.read_excel(LCA_sor_RCP19_DAC_LR_con, sheet_name = key)

# print(LCA_sor_RCP19_DAC_con_dict)

LCA_sor_RCP19_DAC_LR_country_dict = {}
for key in LCA_sor_RCP19_DAC_LR_con_dict.keys():
    tem = LCA_sor_RCP19_DAC_LR_con_dict[key]
    tem['Sorbent_HP_US'] = tem['Sorbent_CO2_capture'] + tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_US'] + tem['Sorbent_operation_heat_pump_US_elec'] + tem['Sorbent_EoL_treat']
    tem['Sorbent_BG_US'] = tem['Sorbent_CO2_capture'] + tem['Sorbent_construction'] + tem['Sorbent_operation_material'] + tem['Sorbent_operation_electriciy_US'] + tem['Sorbent_operation_heat_biogas'] + tem['Sorbent_EoL_treat']

    tem['Sorbent_CO2_capture_HP_US'] = tem['Sorbent_CO2_capture']
    tem['Sorbent_construction_HP_US'] = tem['Sorbent_construction']
    tem['Sorbent_operation_material_HP_US'] = tem['Sorbent_operation_material']
    tem['Sorbent_operation_electriciy_HP_US'] = tem['Sorbent_operation_electriciy_US']
    tem['Sorbent_operation_heat_HP_US'] = tem['Sorbent_operation_heat_pump_US_elec']
    tem['Sorbent_EoL_treat_HP_US'] = tem['Sorbent_EoL_treat']
    
    tem['Sorbent_CO2_capture_BG_US'] = tem['Sorbent_CO2_capture']
    tem['Sorbent_construction_BG_US'] = tem['Sorbent_construction']
    tem['Sorbent_operation_material_BG_US'] = tem['Sorbent_operation_material']
    tem['Sorbent_operation_electriciy_BG_US'] = tem['Sorbent_operation_electriciy_US']
    tem['Sorbent_operation_heat_BG_US'] = tem['Sorbent_operation_heat_biogas']
    tem['Sorbent_EoL_treat_BG_US'] = tem['Sorbent_EoL_treat']
    
    tem_country_total = tem[['Unnamed: 0', 
                             'Sorbent_CO2_capture_HP_US', 'Sorbent_construction_HP_US', 'Sorbent_operation_material_HP_US', 'Sorbent_operation_electriciy_HP_US', 'Sorbent_operation_heat_HP_US', 'Sorbent_EoL_treat_HP_US',  
                             'Sorbent_CO2_capture_BG_US', 'Sorbent_construction_BG_US', 'Sorbent_operation_material_BG_US', 'Sorbent_operation_electriciy_BG_US', 'Sorbent_operation_heat_BG_US', 'Sorbent_EoL_treat_BG_US']].copy()
    
    tem_country_total = tem_country_total.set_index(['Unnamed: 0'])
    LCA_sor_RCP19_DAC_LR_country_dict[key[0:4]] = tem_country_total

#LCA_sor_RCP19_DAC_country_dict
LCA_sor_contribution_summary_table = pd.concat(LCA_sor_RCP19_DAC_LR_country_dict ,axis=0)
LCA_sor_contribution_summary_table.index.names = ["Year", 'Impact category']


# HP_US
LCA_sor_contribution_HP_US_table = LCA_sor_contribution_summary_table[['Sorbent_CO2_capture_HP_US', 'Sorbent_construction_HP_US', 'Sorbent_operation_material_HP_US', 'Sorbent_operation_electriciy_HP_US', 'Sorbent_operation_heat_HP_US', 'Sorbent_EoL_treat_HP_US']]
LCA_sor_contribution_HP_US_table = LCA_sor_contribution_HP_US_table.reset_index()
LCA_sor_contribution_HP_US_table.columns=['Year','Impact_category','0_CO2_capture', '1_Construction','2_Op_material','3_Op_electricity', '4_Op_heat', '5_EoL_treat']
LCA_sor_contribution_HP_US_table_gather = LCA_sor_contribution_HP_US_table.melt(id_vars=['Year', 'Impact_category'], 
                                                                                value_vars=['0_CO2_capture', '1_Construction', '2_Op_material', '3_Op_electricity', '4_Op_heat', '5_EoL_treat'],
                                                                                var_name = 'process', 
                                                                                value_name = 'value')
LCA_sor_contribution_HP_US_table_gather['Tech_heat_region'] = 'Sor_HP_US'
LCA_sor_contribution_HP_US_table_gather = LCA_sor_contribution_HP_US_table_gather[list(('Year', 'Impact_category', 'Tech_heat_region', 'process', 'value'))]
LCA_sor_contribution_HP_US_table_gather = LCA_sor_contribution_HP_US_table_gather.sort_values(by=['Year', 'Impact_category', 'Tech_heat_region', 'process'])
#LCA_sor_contribution_HP_US_table_gather


# BG_US
LCA_sor_contribution_BG_US_table = LCA_sor_contribution_summary_table[['Sorbent_CO2_capture_BG_US', 'Sorbent_construction_BG_US', 'Sorbent_operation_material_BG_US', 'Sorbent_operation_electriciy_BG_US', 'Sorbent_operation_heat_BG_US', 'Sorbent_EoL_treat_BG_US']]
LCA_sor_contribution_BG_US_table = LCA_sor_contribution_BG_US_table.reset_index()
LCA_sor_contribution_BG_US_table.columns=['Year','Impact_category','0_CO2_capture', '1_Construction','2_Op_material','3_Op_electricity', '4_Op_heat', '5_EoL_treat']
LCA_sor_contribution_BG_US_table_gather = LCA_sor_contribution_BG_US_table.melt(id_vars=['Year', 'Impact_category'], 
                                                                                value_vars=['0_CO2_capture', '1_Construction', '2_Op_material', '3_Op_electricity', '4_Op_heat', '5_EoL_treat'],
                                                                                var_name = 'process', 
                                                                                value_name = 'value')
LCA_sor_contribution_BG_US_table_gather['Tech_heat_region'] = 'Sor_BG_US'
LCA_sor_contribution_BG_US_table_gather = LCA_sor_contribution_BG_US_table_gather[list(('Year', 'Impact_category', 'Tech_heat_region', 'process', 'value'))]
LCA_sor_contribution_BG_US_table_gather = LCA_sor_contribution_BG_US_table_gather.sort_values(by=['Year', 'Impact_category', 'Tech_heat_region', 'process'])
#LCA_sor_contribution_BG_US_table_gather


LCA_sor_RCP19_DAC_LR_contribution_all_summary_table = pd.concat([LCA_sor_contribution_HP_US_table_gather,
                                                LCA_sor_contribution_BG_US_table_gather], axis=0)
LCA_sor_RCP19_DAC_LR_contribution_all_summary_table

In [ ]:
# Export the results to excel

LCA_sol_RCP19_DAC_contribution_all_summary_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_RCP19_DAC_contribution_all_summary_table.xlsx', index = False) 
LCA_sol_RCP19_DAC_LR_contribution_all_summary_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sol_RCP19_DAC_LR_contribution_all_summary_table.xlsx', index = False) 
LCA_sor_RCP19_DAC_contribution_all_summary_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_RCP19_DAC_contribution_all_summary_table.xlsx', index = False)
LCA_sor_RCP19_DAC_LR_contribution_all_summary_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\LCA_result\LCA_sor_RCP19_DAC_LC_contribution_all_summary_table.xlsx', index = False) 